### 人类活动识别

In [ ]:
# 数据预处理
from torchvision import transforms
from model.Normalization import *

transform=transforms.Compose([
    RobustNorm(-7.9, 8.2),
])

In [ ]:
# 加载数据集
from model.RFID_Dataset import RFID_Dataset
from torch.utils.data import ConcatDataset

train_dir = r"data\RFID_ENV\dataset\train"
eval_dir = r"data\RFID\dataset\eval"

train_dataset = RFID_Dataset(
    train_dir,
    T=32,
    step=1,
    transform=transform,
)

eval_dataset = RFID_Dataset(
    eval_dir,
    T=32,
    step=1,
    transform=transform,
)

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 加载数据集
from model.RFID_Dataset import RFID_Dataset
from utils.DatasetUtils import DatasetUtils

all_dir=r"data\RFID_ENV\dataset\all"

all_dataset = RFID_Dataset(
    all_dir,
    T=12,
    step=1,
    transform=transform,
)

dataset_utils=DatasetUtils()
train_dataset, eval_dataset = dataset_utils.split_dataset(all_dataset)


print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网
from torchkeras import summary
from model.ClassifyNet.CNNClassifyNet import CNNClassifyNet as ClassifyNet
from model.ModelWorker.ClassifyModelWorker import ClassifyModelWorker

input_shape=(1,12,12)
model=ClassifyNet(
    input_shape=input_shape,
    num_classes=6
)

model_worker=ClassifyModelWorker(model)

print(f"{input_shape=}")
model_info=summary(model,input_shape=input_shape)

In [ ]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torchmetrics import Accuracy
from model.LightningModel.ClassifyPLModel import ClassifyPLModel

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16)

loss = nn.CrossEntropyLoss()
metrics= Accuracy(task="multiclass", num_classes=6)


# 构建PLModel
pl_model = ClassifyPLModel(
    model,
    loss,
    metrics
)

In [ ]:
# 模型训练

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping

trainer = pl.Trainer(
    max_epochs=None,
    logger=True,
    callbacks=[
        ModelCheckpoint(save_weights_only=True),
        EarlyStopping(monitor="val/loss", patience=5),
    ],
)

trainer.fit(
    pl_model,
    train_loader,
    eval_loader,
)

In [ ]:
# 模型评估

model_worker.evaluate(
    eval_loader=eval_loader,
    criterion=loss,
)

In [ ]:
# 评价指标

from torchmetrics import MetricCollection, Accuracy, F1Score,ConfusionMatrix
from utils.ConfigUtils import get_classes

classes=get_classes(r"data\RFID_ENV\data.yml")
num_classes = len(classes)

res=model_worker.execute_metric(
    train_loader, 
    MetricCollection([
        Accuracy(task="multiclass", num_classes=num_classes),
        F1Score(task="multiclass", num_classes=num_classes),
        ConfusionMatrix(task="multiclass", num_classes=num_classes,normalize="true")
    ]),
)

for item in res.items():
    print(f"{item[0]}:\n\t {item[1]}")

In [ ]:
from utils.DataUtils.Visualization import plot_confusion_matrix

confusion_matrix=res["MulticlassConfusionMatrix"].cpu()
plot_confusion_matrix(
    confusion_matrix,
    class_names=classes.values(),
    is_percentage=True,
    title="train confusion matrix",
)

In [ ]:
# 保存模型
model_worker.save('./output/HAR_ENV/HAR.pth')

In [ ]:
# 加载模型
model_worker.load('./output/HAR_ENV/HAR.pth')

In [ ]:
from torch.utils.data import DataLoader


test_dataset=RFID_Dataset(
    r"./output/merge",
    T=32,
    step=32,
    # transform=transform,
)
print(f"测试集的数据个数: {len(test_dataset)}")

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=8)
model_worker.evaluate(
    eval_loader=test_loader,
    criterion=loss,
)

In [ ]:
# 模型预测

used_dataset = eval_dataset
indexs,inputs,labels = dataset_utils.select_simple(used_dataset,count=4)
preds,acc = model_worker.predict(inputs=inputs,labels=labels)

print(f"{inputs.shape=}")
print(f"{acc=}")
print(f"indexs:{indexs}")
print(f"labels:{labels}")
print(f"preds :{preds}")